In [1]:
import pandas as pd
# anno_1 = '../Human_anno/humananno_res/lsy_scene.xlsx'
# anno_2 = '../Human_anno/humananno_res/lsy_scene.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','scene'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','scene'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

In [2]:
import numpy as np
    
def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr

def rank_basescore(base_score, draw_ratio):
    # 计算最大值和最小值的差值
    score_range = max(base_score) - min(base_score)
    # 计算平序的阈值
    draw_gap = draw_ratio * score_range

    # 对列表进行排序并保留原始索引
    indexed_scores = list(enumerate(base_score))
    indexed_scores.sort(key=lambda x: x[1])

    # 处理平序
    ranks = [0] * len(base_score)
    current_rank = 1
    for i in range(len(indexed_scores)):
        if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:
            ranks[indexed_scores[i][0]] = current_rank
        else:
            current_rank = i + 1
            ranks[indexed_scores[i][0]] = current_rank

    return ranks

In [3]:
dimension = 'scene'

In [4]:
import json
jsonpath = '../Human_anno/{}.json'.format(dimension)
with open(jsonpath,'r') as f:
    oc = json.load(f)

# history ="../GPT4o_eval_results/{}_gpt4eval_results.json".format(dimension)
# with open(history,'r') as f:
#     gpt4o_eval_history = json.load(f)

In [5]:
# gptvsanno1_spearman = 0
# gptvsanno2_spearman = 0
# anno1vsanno2_spearman = 0
# gptvsannomean = 0
# badeval = []

# for i in range(length):
#     if i % 3 == 0:
#         gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 = human_anno[:,0]
#         anno2 = human_anno[:,1]
#         annomean = (anno1 + anno2)/2
#     else:
#         gpt4o_eval_rs+= np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 += human_anno[:,0]
#         anno2 += human_anno[:,1]
#         annomean += (anno1 + anno2)/2

#     if i % 3 == 2:
#         gptvsanno1_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno1)
#         gptvsanno2_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno2)
#         anno1vsanno2_spearman += calculate_spearman_manual(anno1,anno2)
#         gptvsannomean += calculate_spearman_manual(gpt4o_eval_rs,annomean)

# gptvsanno1_spearman = 3*gptvsanno1_spearman/length
# gptvsanno2_spearman = 3*gptvsanno2_spearman/length
# anno1vsanno2_spearman = 3*  anno1vsanno2_spearman/length
# gptvsannomean = 3*gptvsannomean/length
# print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman)
# print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman)
# print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman)
# print("GPT vs AnnoMean Spearman: ",gptvsannomean)

In [6]:
models = ['cogvideox5b','gen3', 'kling','videocrafter2', 'pika', 'show1', 'lavie']
# models = ['videocrafter2', 'pika', 'show1', 'lavie']
# models = ['cogvideox5b','gen3', 'kling']
idexls = []
for i in range(0,len(oc),3):
    idexls.append(i)
# for i in range(1,len(oc),3):
#     idexls.append(i)
length = len(idexls)

In [7]:
flag = 0
iternum = 0
while(flag == 0):
    anno1s = []
    anno2s = []
    anno3s = []
    anno4s = []
    iternum += 1
    gptvsanno1_spearman = np.zeros(length)
    gptvsanno2_spearman = np.zeros(length)
    gptvsanno3_spearman = np.zeros(length)
    gptvsanno4_spearman = np.zeros(length)
    gptvsannomean_spearman = np.zeros(length)
    baselinevsanno1_spearman = np.zeros(length)
    baselinevsanno2_spearman = np.zeros(length)
    baselinevsanno3_spearman = np.zeros(length)
    baselinevsanno4_spearman = np.zeros(length)
    baselinevsannomean_spearman = np.zeros(length)

    gptscore = np.zeros([len(models)])
    anno1score = np.zeros([len(models)])
    anno2score = np.zeros([len(models)])
    anno3score = np.zeros([len(models)])
    anno4score = np.zeros([len(models)])
    annomeanscore =np.zeros([len(models)])
    baseline_rank = np.zeros([7])
    badeval = []

    for j in range(length):
        i = idexls[j]
        gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
        human_anno = np.array(list(oc[i]['human_anno'].values()))
        baseline_score = np.array(list(oc[i]['baseline_score'].values()))
        anno1 = human_anno[:,0]
        anno2 = human_anno[:,1]
        anno3 = human_anno[:,2]
        anno4 = human_anno[:,3]


        anno1s.append(anno1)
        anno2s.append(anno2)
        anno3s.append(anno3)
        anno4s.append(anno4)

        annomean = (anno1 + anno2 +anno3+anno4)/4


        gptscore += gpt4o_eval_rs
        anno1score += anno1
        anno2score += anno2
        anno3score += anno3
        anno4score += anno4
        annomeanscore += annomean
        baseline_rank = rank_basescore(baseline_score, 0.165)

        gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
        gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
        gptvsanno3 = calculate_spearman_manual(gpt4o_eval_rs,anno3)
        gptvsanno4 = calculate_spearman_manual(gpt4o_eval_rs,anno4)
        gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
        baselinevsanno1 = calculate_spearman_manual(baseline_rank,anno1)
        baselinevsanno2 = calculate_spearman_manual(baseline_rank,anno2)
        baselinevsanno3 = calculate_spearman_manual(baseline_rank,anno3)
        baselinevsanno4 = calculate_spearman_manual(baseline_rank,anno4)
        baselinevsannomean = calculate_spearman_manual(baseline_rank,annomean)

        # j = i
        gptvsanno1_spearman[j] = gptvsanno1
        gptvsanno2_spearman[j] = gptvsanno2
        gptvsanno3_spearman[j] = gptvsanno3
        gptvsanno4_spearman[j] = gptvsanno4
        gptvsannomean_spearman[j] = gptvsannomean
        baselinevsanno1_spearman[j] = baselinevsanno1
        baselinevsanno2_spearman[j] = baselinevsanno2
        baselinevsanno3_spearman[j] = baselinevsanno3
        baselinevsanno4_spearman[j] = baselinevsanno4
        baselinevsannomean_spearman[j] = baselinevsannomean

    gptscore = gptscore/length
    anno1score = anno1score/length
    anno2score = anno2score/length
    anno3score = anno3score/length
    anno4score = anno4score/length
    annomeanscore = annomeanscore/length

    print("{} iter max gap in gpt".format(iternum))
    print(np.max(np.abs(gptscore - anno1score)),np.argmax(np.abs(gptscore - anno1score)))
    print(np.max(np.abs(gptscore - anno2score)),np.argmax(np.abs(gptscore - anno2score)))
    print(np.max(np.abs(gptscore - anno3score)),np.argmax(np.abs(gptscore - anno3score)))

    # if np.max(np.abs(gptscore - anno1score) ) < 0.1 or np.max(np.abs(gptscore - anno2score)) < 0.1 or np.max(np.abs(gptscore - anno3score)) < 0.1:
    flag = 1

    if iternum%10==0:
        print("GPT average score: ",gptscore)
        print("Anno1 average score: ",anno1score)
        print("Anno2 average score: ",anno2score)
        print("Anno3 average score: ",anno3score)
        print("Anno4 average score: ",anno4score)

print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("Anno3 average score: ",anno3score)
print("Anno4 average score: ",anno4score)
print("AnnoMean average score: ",annomeanscore)

print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("GPT vs Anno3 Spearman: ",gptvsanno3_spearman.mean())
print("GPT vs Anno4 Spearman: ",gptvsanno4_spearman.mean())
print("Average GPT vs Anno Spearman: ",(gptvsanno1_spearman.mean()+gptvsanno2_spearman.mean()+gptvsanno3_spearman.mean())/3)
print("GPT vs AnnoMean Spearman: ",gptvsannomean_spearman.mean())
print("Baseline vs Anno1 Spearman: ",baselinevsanno1_spearman.mean())
print("Baseline vs Anno2 Spearman: ",baselinevsanno2_spearman.mean())
print("Baseline vs Anno3 Spearman: ",baselinevsanno3_spearman.mean())
print("Baseline vs Anno4 Spearman: ",baselinevsanno4_spearman.mean())
print("Average Baseline vs Anno Spearman: ",(baselinevsanno1_spearman.mean()+baselinevsanno2_spearman.mean()+baselinevsanno3_spearman.mean())/3)
print("Baseline vs AnnoMean Spearman: ",baselinevsannomean_spearman.mean())




1 iter max gap in gpt
0.44186046511627897 6
0.4767441860465116 6
0.6162790697674418 3
GPT average score:  [2.93023256 2.91860465 2.79069767 2.53488372 2.39534884 2.5
 2.18604651]
Anno1 average score:  [2.90697674 2.93023256 2.8255814  2.73255814 2.47674419 2.6744186
 2.62790698]
Anno2 average score:  [3.         3.         2.93023256 2.84883721 2.68604651 2.8255814
 2.6627907 ]
Anno3 average score:  [2.72093023 2.95348837 2.89534884 1.91860465 1.80232558 1.91860465
 2.06976744]
Anno4 average score:  [2.74418605 2.94186047 2.90697674 2.1744186  2.15116279 2.19767442
 2.30232558]
AnnoMean average score:  [2.84302326 2.95639535 2.88953488 2.41860465 2.27906977 2.40406977
 2.41569767]
GPT vs Anno1 Spearman:  0.7086794019933554
GPT vs Anno2 Spearman:  0.7153239202657807
GPT vs Anno3 Spearman:  0.641611295681063
GPT vs Anno4 Spearman:  0.6248961794019933
Average GPT vs Anno Spearman:  0.6885382059800663
GPT vs AnnoMean Spearman:  0.6499169435215946
Baseline vs Anno1 Spearman:  0.142026578073

C:\Users\LiGe\AppData\Local\Temp\ipykernel_12136\4281206334.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  score_range = max(base_score) - min(base_score)
C:\Users\LiGe\AppData\Local\Temp\ipykernel_12136\4281206334.py:38: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  indexed_scores.sort(key=lambda x: x[1])
C:\Users\LiGe\AppData\Local\Temp\ipykernel_12136\4281206334.py:44: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:


In [8]:
num_human_anno = 3
spearmans =np.zeros([num_human_anno,num_human_anno,length])
annos = [anno1s,anno2s,anno3s]
for j in range(length):
    i = idexls[j]
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    for k in range(num_human_anno):
        for l in range(num_human_anno):
            spearmans[k,l,j] = calculate_spearman_manual(annos[k][j],annos[l][j])
average_spearmans = spearmans.mean(axis=2)

print(average_spearmans)
print((average_spearmans.sum()-num_human_anno)/2/num_human_anno)

[[1.         0.80938538 0.65178571]
 [0.80938538 1.         0.6255191 ]
 [0.65178571 0.6255191  1.        ]]
0.695563399778516
